Because neither Ensembl nor GENCODE include tRNA sequences in their transcripts file. We need to get tRNA sequences first.

First get the tRNA sequences:

In [1]:
!bedtools getfasta -fi temporary_data/GRCh38.primary_assembly.genome.fa -bed <(awk -F'\t' '$3=="tRNA"' temporary_data/gencode.v49.tRNAs.gff3 | awk -F'\t' 'BEGIN{OFS="\t"} {print $1, $4-1, $5, $9, ".", $7}') -name > temporary_data/gencode.v49.tRNAs.fa

WARNING. chromosome (MU273365.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (MU273333.1) was not found in the FASTA file. Skipping.
WARNING. chr

Next, we rename the FASTA headers to the GENCODE format.

In [2]:
#!/usr/bin/env python3
import re
import sys

in_fa = sys.argv[1] if len(sys.argv) > 1 else "-"
out_fa = sys.argv[2] if len(sys.argv) > 2 else "-"

# Set these if you want synthetic IDs that "look like" stable accessions.
# If you prefer to keep raw numeric IDs, set both prefixes to "".
TX_PREFIX = "TRNST"   # e.g., TRNST00033323.1  (synthetic transcript)
GN_PREFIX = "TRNSG"   # e.g., TRNSG00033323.1  (synthetic gene)
ADD_VERSION = True    # append ".1"

def parse_attrs(header_left: str) -> dict:
    # header_left looks like: ID=33323;gene_id=33323;transcript_id=33323;gene_type=Pseudo_tRNA;...
    attrs = {}
    for part in header_left.split(";"):
        if "=" in part:
            k, v = part.split("=", 1)
            attrs[k] = v
    return attrs

def fmt_id(prefix: str, raw: str) -> str:
    if prefix:
        # zero-pad numeric ids for nicer sorting; fallback to raw if not numeric
        if raw.isdigit():
            raw2 = f"{int(raw):08d}"
        else:
            raw2 = raw
        raw = f"{prefix}{raw2}"
    if ADD_VERSION and not re.search(r"\.\d+$", raw):
        raw = raw + ".1"
    return raw

def flush(rec_header, seq_chunks, out_handle):
    if rec_header is None:
        return
    seq = "".join(seq_chunks).replace(" ", "").replace("\n", "")
    length = len(seq)

    # Split at ::chr... if present
    left = rec_header
    if "::" in rec_header:
        left = rec_header.split("::", 1)[0]

    attrs = parse_attrs(left)
    gene_id = attrs.get("gene_id", attrs.get("ID", "NA"))
    tx_id   = attrs.get("transcript_id", attrs.get("ID", "NA"))
    gene_type = "tRNA"
    gene_name = attrs.get("gene_name", gene_id)
    tx_name   = attrs.get("transcript_name", tx_id)

    out_tx_id = fmt_id(TX_PREFIX, tx_id) if TX_PREFIX or ADD_VERSION else tx_id
    out_gn_id = fmt_id(GN_PREFIX, gene_id) if GN_PREFIX or ADD_VERSION else gene_id

    new_header = f">{out_tx_id}|{out_gn_id}|-|-|{tx_name}|{gene_name}|{length}|{gene_type}|"
    out_handle.write(new_header + "\n")
    # Wrap sequence at 60 like many GENCODE FASTAs
    for i in range(0, len(seq), 60):
        out_handle.write(seq[i:i+60] + "\n")

In [3]:
inp = open('temporary_data/gencode.v49.tRNAs.fa', "r")
out = open('temporary_data/gencode.v49.tRNAs.formated.fa', "w")

rec_header = None
seq_chunks = []

for line in inp:
    line = line.rstrip("\n")
    if line.startswith(">"):
        flush(rec_header, seq_chunks, out)
        rec_header = line[1:]  # strip ">"
        seq_chunks = []
    else:
        if line:
            seq_chunks.append(line.strip())

flush(rec_header, seq_chunks, out)

inp.close()
out.close()

Concat the tRNA fasta file to the GENCODE transcripts file.

In [4]:
!cat temporary_data/gencode.v49.transcripts.fa temporary_data/gencode.v49.tRNAs.formated.fa > temporary_data/gencode.v49.transcripts_plus_tRNA.fa